!git clone https://github.com/trucomanx/cnn_face_emotion.git
!mkdir -p OUTPUTS/cnn_face_emotion/cross-validation
!wget https://www.dropbox.com/s/23tutsg2m9o78bp/fer2013.zip
!unzip -n fer2013.zip
!rm fer2013.zip

In [1]:
import os, sys, platform

# Variables

In [2]:
## Seed for the random variables
seed_number=0;



## Training hyperparameters
EPOCAS=50;
BATCH_SIZE=16;

## Model of network
#model_type  = 'mobilenet_v3';
model_type = 'efficientnet_b3'
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';
#model_type = 'custom1'
#model_type = 'custom_inception'
#model_type = 'custom_residual1'
#model_type = 'custom_dense1'

DATASET_NAME = 'mcfer_v1.0';#'fer2013';


dataset_labels_file = 'training_labels.csv';


# Bibliotecas externas

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

2023-05-15 17:40:58.494223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 17:40:59.614810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fernando/anaconda3/lib/
2023-05-15 17:40:59.614980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fernando/anaconda3/lib/
2023-05-15 17:40:59.614985: W tensorflow/compiler/tf2tensorrt/utils/

# Biblioteca local

In [4]:
if os.getenv("COLAB_RELEASE_TAG"):
    sys.path.append('cnn_face_emotion/library');
else:
    sys.path.append('library');

# Uso de GPU

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2023-05-15 17:41:02.354568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 17:41:02.607254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 17:41:02.607354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Endereços

In [6]:

## Dataset 
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir_raw = './';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir_raw = '/mnt/c/Dados/Fernando/DATASET';
    else:
        dataset_base_dir_raw = '/media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET';
else:
    dataset_base_dir_raw     = 'C:\\Dados\\Fernando\\DATASET';

if DATASET_NAME=='fer2013':
    RELATIVE_DIR_TRAIN=os.path.join(DATASET_NAME,'archive','train');
    RELATIVE_DIR_TEST =os.path.join(DATASET_NAME,'archive','test');
elif DATASET_NAME=='affectnet':
    RELATIVE_DIR_TRAIN=os.path.join('AffectNet-Sample','input','affectnetsample','train_class');
    RELATIVE_DIR_TEST =os.path.join('AffectNet-Sample','input','affectnetsample','val_class');
elif DATASET_NAME=='mcfer_v1.0':
    RELATIVE_DIR_TRAIN=os.path.join(DATASET_NAME,'archive','train');
    RELATIVE_DIR_TEST =os.path.join(DATASET_NAME,'archive','test');
else:
    print('Error in the dataset name:',DATASET_NAME);
    exit();
'''    
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir     = 'fer2013/archive/train';
        dataset_base_test_dir= 'fer2013/archive/test';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir     = '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/test';
    else:
        dataset_base_dir     = '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test';
else:
    dataset_base_dir     = 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\train';
    dataset_base_test_dir= 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\test';
'''

dataset_base_dir = os.path.join(dataset_base_dir_raw,RELATIVE_DIR_TRAIN);
dataset_base_test_dir = os.path.join(dataset_base_dir_raw,RELATIVE_DIR_TEST);

print('dataset_base_dir:',dataset_base_dir)
print('dataset_base_test_dir:',dataset_base_test_dir)

## Output
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        output_base_dir_raw = 'OUTPUTS';
    elif 'microsoft-standard' in platform.uname().release:
        output_base_dir_raw = '/mnt/c/Dados/Fernando/OUTPUTS';
    else:
        output_base_dir_raw = '/media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS';
else:
    output_base_dir_raw = 'C:\\Dados\\Fernando\\OUTPUTS';

    
RELATIVE_DIR=os.path.join('cnn_face_emotion_'+DATASET_NAME,'training_holdout');
output_base_dir = os.path.join(output_base_dir_raw,RELATIVE_DIR); 

print('output_base_dir:',output_base_dir)

dataset_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/train
dataset_base_test_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/test
output_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/cnn_face_emotion_mcfer_v1.0/training_holdout


# Print vars

In [7]:
print('model_type:',model_type)
print('    epochs:',EPOCAS)
print('batch-size:',BATCH_SIZE)

model_type: efficientnet_b3
    epochs: 50
batch-size: 16


# Set seed of random variables


In [8]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [9]:
# Load filenames and labels
train_data = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(train_data)
# Setting labels
Y = train_data[['label']];
L=np.shape(Y)[0];


                                    filename      label
0      disgusted/awful/affectnet_0036766.png  disgusted
1                         sad/crying/im5.png        sad
2       fearful/attention-fearful/im32_1.png    fearful
3                neutral/attention/im2_1.png    neutral
4             angry/angry-disgusted/im19.png      angry
...                                      ...        ...
24231         happy/light-smile/im4724_1.png      happy
24232           happy/light-smile/im4742.png      happy
24233           happy/light-smile/im4809.png      happy
24234           happy/light-smile/im5535.png      happy
24235           happy/light-smile/im5545.png      happy

[24236 rows x 2 columns]


# Setting the cross-validation split


In [10]:
from sklearn.model_selection import train_test_split
    

training_data, validation_data = train_test_split(train_data, test_size=0.2,shuffle=True, stratify=Y)



# Data augmentation configuration

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg    = ImageDataGenerator(rescale=1./255,
                            rotation_range = 10,
                            width_shift_range= 0.07,
                            height_shift_range= 0.07,
                            horizontal_flip=True,
                            shear_range=1.25,
                            zoom_range = [0.75, 1.25],
                            validation_split=0.25
                            )

#idg_val= ImageDataGenerator(rescale=1./255 )

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [12]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,'holdout_'+model_type);

try: 
    os.makedirs(output_dir) 
except: 
    pass


# Create new model

In [13]:
import lib_model as mpp

model, target_size = mpp.create_model('',model_type=model_type);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


2023-05-15 17:41:05.874179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 17:41:05.875048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 17:41:05.875222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 17:41:05.875299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Loaded layer with efficientnet_b3
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              10783528  
                                                                 
 dense (Dense)               (None, 32)                49184     
                                                                 
 dense_1 (Dense)             (None, 7)                 231       
                                                                 
Total params: 10,832,943
Trainable params: 10,745,647
Non-trainable params: 87,296
_________________________________________________________________


# Defining directories

In [14]:

train_data_generator = idg.flow_from_dataframe(training_data, 
                                               directory = dataset_base_dir,
                                               target_size=target_size,
                                               x_col = "filename", 
                                               y_col = "label",
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical",
                                               shuffle = True);

valid_data_generator  = idg.flow_from_dataframe(validation_data, 
                                                    directory = dataset_base_dir,
                                                    target_size=target_size,
                                                    x_col = "filename", 
                                                    y_col = "label",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle = True);

test_data_generator  = idg_test.flow_from_directory(dataset_base_test_dir, 
                                                    target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True
                                                    );

Found 19388 validated image filenames belonging to 7 classes.
Found 4848 validated image filenames belonging to 7 classes.
Found 6709 images belonging to 7 classes.


# Train and validation


In [ ]:
import matplotlib.pyplot as plt


#STEPS_BY_EPOCHS=len(train_data_generator);

# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','categorical_crossentropy'])

# CREATE CALLBACKS
best_model_file=os.path.join(output_dir,'model.h5');
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                save_weights_only=True,
                                                monitor='val_categorical_crossentropy', 
                                                save_best_only=True, 
                                                verbose=1);

log_dir = os.path.join(output_dir,"logs","fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# There can be other callbacks, but just showing one because it involves the model name
# This saves the best model
# FIT THE MODEL
history = model.fit(train_data_generator,
                    #steps_per_epoch=STEPS_BY_EPOCHS,
                    epochs=EPOCAS,
                    validation_data=valid_data_generator,
                    callbacks=[checkpoint,tensorboard_callback],
                    verbose=1
                   );


mpp.save_model_history(history,os.path.join(output_dir,"historical.csv"));


Epoch 1/50


2023-05-15 17:41:38.871758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-05-15 17:41:39.535857: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-15 17:41:40.857884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-15 17:41:40.860121: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x922136d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-15 17:41:40.860136: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-05-15 17:41:40.863301: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-15 17:41:40.908514: I

1212/1212 [==============================] - ETA: 0s - loss: 1.4947 - categorical_accuracy: 0.5336 - categorical_crossentropy: 1.3094
Epoch 1: val_categorical_crossentropy improved from inf to 1.14356, saving model to /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/cnn_face_emotion_mcfer_v1.0/training_holdout/holdout_efficientnet_b3/model.h5
1212/1212 [==============================] - 466s 347ms/step - loss: 1.4947 - categorical_accuracy: 0.5336 - categorical_crossentropy: 1.3094 - val_loss: 1.3307 - val_categorical_accuracy: 0.5963 - val_categorical_crossentropy: 1.1436
Epoch 2/50
1212/1212 [==============================] - ETA: 0s - loss: 1.2587 - categorical_accuracy: 0.6104 - categorical_crossentropy: 1.0726
Epoch 2: val_categorical_crossentropy improved from 1.14356 to 1.05823, saving model to /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/cnn_face_emotion_mcfer_v1.0/training_holdout/holdout_efficientnet_b3/model.h5
1212/1212 [==============================] - 4

# Evaluate best model

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);

results = model.evaluate(test_data_generator)
results = dict(zip(model.metrics_names,results))
print(results,"\n\n");

with open(os.path.join(output_dir,"results_testing.txt"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));

tf.keras.backend.clear_session()


In [ ]:

tmp_name='modelo_'+model_type+'.h5';

os.rename(best_model_file,os.path.join(output_dir,tmp_name));